In [8]:
# Import necessary libraries
import pandas as pd
import json
import string 

# Load the CSV dataset
df = pd.read_csv("semi_strut.csv")

# Display the first few rows of the dataset
df.head(3)

,Document ID,Content
0,1,"{\r\n ""title"": ""Introduction to Python"",\r\n..."
1,2,"{\r\n ""title"": ""Data Analysis with Pandas"",\..."
2,3,"{\r\n ""title"": ""Web Development with Flask"",..."


In [44]:
# Tokenization function to extract terms from the JSON-like content
# Remember to exact both 
# 1 .Extract terms from various fields (title, author)
def tokenize_content(content):
    content_dict = json.loads(content)
    terms = []
    
    # Extract terms from various fields (title, author)
    terms.extend(content_dict.get("title", "").split())
    terms.extend(content_dict.get("author", "").split())

    #extract terms from sections title and content
    sections = content_dict.get("sections",[])
    for section in sections:
        terms.extend(section.get("title","").split())
        terms.extend(section.get("content","").split())
    terms = [term.lower().strip(string.punctuation) for term in terms]    
    return terms
tokenize_content(df["Content"][0])
# 2. apply to all row in panda df , by create new column "Terms"
df["Terms"] = df["Content"].apply(tokenize_content)
df[["Document ID", "Content","Terms"]]

,Document ID,Content,Terms
0,1,"{\r\n ""title"": ""Introduction to Python"",\r\n...","[introduction, to, python, john, doe, getting,..."
1,2,"{\r\n ""title"": ""Data Analysis with Pandas"",\...","[data, analysis, with, pandas, jane, smith, in..."
2,3,"{\r\n ""title"": ""Web Development with Flask"",...","[web, development, with, flask, mike, johnson,..."
3,4,"{\r\n ""title"": ""Machine Learning with Scikit...","[machine, learning, with, scikit-learn, emily,..."
4,5,"{\r\n ""title"": ""Data Visualization with Matp...","[data, visualization, with, matplotlib, robert..."


In [45]:
# 4. Implement a preprocessing function that converts terms to lowercase, removes punctuation, and removes common stop words.
    # Create another new column "Terms_preprocessed"
def preprocess_terms(terms):
    # Define a set of common stop words
    stop_words = set([
        "a", "an", "the", "and", "is", "in", "it", "to", "of", "for", "on", "with", "as"
    ])
    
    # Remove punctuation and convert to lowercase
    terms = [term.lower().strip(string.punctuation) for term in terms]
    
    # Remove stop words
    terms = [term for term in terms if term not in stop_words]
    
    return terms

# your code here....
unique_terms = {term for doc in df["Terms"] for term in doc}
unique_terms


{'a',
 'analysis',
 'and',
 'are',
 'artificial',
 'basic',
 'be',
 'can',
 'clark',
 'core',
 'created',
 'creating',
 'data',
 'dataframes',
 'davis',
 'defines',
 'development',
 'doe',
 'easy',
 'emily',
 'flask',
 'for',
 'framework',
 'functions',
 'getting',
 'in',
 'intelligence',
 'introduction',
 'is',
 'jane',
 'john',
 'johnson',
 'language',
 'learning',
 'library',
 'lightweight',
 'machine',
 'matplotlib',
 'mike',
 'of',
 'pandas',
 'patterns',
 'plots',
 'popular',
 'programming',
 'python',
 'robert',
 'routing',
 'scikit-learn',
 'smith',
 'started',
 'structure',
 'subfield',
 'supervised',
 'syntax',
 'to',
 'type',
 'understand',
 'url',
 'using',
 'various',
 'versatile',
 'views',
 'visualization',
 'web',
 'with'}

In [46]:
# Initialize an empty inverted index dictionary
# Build the inverted index
# Initialize an empty inverted index dictionary
inverted_index = {}

# Build the inverted index
for index, row in df.iterrows():
    document_id = row["Document ID"]
    terms = row["Terms"]
    
    # Update the inverted index with terms and document IDs
    for term in terms:
        if term not in inverted_index:
            inverted_index[term] = set()
        inverted_index[term].add(document_id)

# Display the inverted index
inverted_index

{'introduction': {1, 2, 4, 5},
 'to': {1, 2, 4, 5},
 'python': {1, 3},
 'john': {1},
 'doe': {1},
 'getting': {1, 3},
 'started': {1, 3},
 'is': {1, 2, 3, 4, 5},
 'a': {1, 2, 3, 4, 5},
 'versatile': {1, 5},
 'programming': {1},
 'language': {1},
 'basic': {1},
 'syntax': {1},
 'easy': {1},
 'understand': {1},
 'data': {2, 5},
 'analysis': {2},
 'with': {2, 3, 4, 5},
 'pandas': {2},
 'jane': {2},
 'smith': {2},
 'popular': {2},
 'library': {2, 5},
 'for': {2, 3, 5},
 'dataframes': {2},
 'are': {2},
 'core': {2},
 'structure': {2},
 'in': {2, 3, 5},
 'web': {3},
 'development': {3},
 'flask': {3},
 'mike': {3},
 'johnson': {3},
 'lightweight': {3},
 'framework': {3},
 'routing': {3},
 'defines': {3},
 'url': {3},
 'patterns': {3},
 'and': {3},
 'views': {3},
 'machine': {4},
 'learning': {4},
 'scikit-learn': {4},
 'emily': {4},
 'davis': {4},
 'subfield': {4},
 'of': {4},
 'artificial': {4},
 'intelligence': {4},
 'supervised': {4},
 'type': {4},
 'visualization': {5},
 'matplotlib': {5

In [48]:
# perform boolean operations on postings lists for Boolean search operations
posting_list = inverted_index['introduction']
posting_list
# 1. "Python" OR "Pandas"
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1 += 1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2 += 1
    return result
    pl_1 = list(inverted_index['Python'])
    pl_2 = list(inverted_index['Pandas'])
    or_postings(pl_1,pl_2)
# 2. "Python" AND "data"
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] == posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result
    pl_1 = list(inverted_index['Python'])
    pl_2 = list(inverted_index['Data'])
    and_postings(pl_1,pl_2)